In [1]:
!pip install torch
!pip install lightning

In [31]:
import torch 
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import lightning as L

In [32]:
from torch.utils.data import DataLoader, TensorDataset

In [33]:
class LSTMbyHand(L.LightningModule):
    def __init__(self):
        super().__init__()

        norm_dist = torch.randn(1)
        zero_tensor = torch.tensor(0).float()

        self.wfs = nn.Parameter(norm_dist, requires_grad=True)
        self.wfi = nn.Parameter(norm_dist, requires_grad=True)
        self.bf = nn.Parameter(zero_tensor, requires_grad=True)

        self.wis1 = nn.Parameter(norm_dist, requires_grad=True)
        self.wii1 = nn.Parameter(norm_dist, requires_grad=True)
        self.bi1 = nn.Parameter(zero_tensor, requires_grad=True)
        
        self.wis2 = nn.Parameter(norm_dist, requires_grad=True)
        self.wii2 = nn.Parameter(norm_dist, requires_grad=True)
        self.bi2 = nn.Parameter(zero_tensor, requires_grad=True)

        self.wos = nn.Parameter(norm_dist, requires_grad=True)
        self.woi = nn.Parameter(norm_dist, requires_grad=True)
        self.bo = nn.Parameter(zero_tensor, requires_grad=True)

    def module(self, inputs, long_memory, short_memory):

        per_long_memory = torch.sigmoid(self.wfs*short_memory + self.wfi*inputs + self.bf)
        prev_long_memory = long_memory * per_long_memory

        perc_long_memory = torch.sigmoid(self.wis1*short_memory + self.wii1*inputs + self.bi1)
        new_long_memory = torch.tanh(self.wis2*short_memory + self.wii2*inputs + self.bi2)

        combined_long_memory = prev_long_memory + (perc_long_memory * new_long_memory)

        new_short_memory = torch.sigmoid(self.wos*short_memory + self.woi*inputs + self.bo)
        combined_short_memory = torch.tanh(combined_long_memory) * new_short_memory

        return ([combined_long_memory, combined_short_memory])
    
    def forward(self, inputs):

        long_memeory = 0
        short_memory = 0
        for i in inputs:
            long_memory, short_memory = self.module(i, long_memeory, short_memory)

        return short_memory
    
    def optimizer(self):

        return Adam(self.parameters())
    
    def training(self, batch, batch_idx):
        input_i, label_i = batch
        pred = self.forward(input_i[0])
        loss = (pred - label_i) **2

        self.log('train_loss', loss)

        if (label_i ==0):
            self.log('out_0', pred)
        else:
            self.log('out_1', pred)

        return loss

In [34]:
model = LSTMbyHand()
preds = model(torch.tensor([0, 0.5, 0.25, 1]).detach())
print(f"predictions: {preds}")

predictions: tensor([-0.0520], grad_fn=<MulBackward0>)


In [ ]:
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

